In [23]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.dummy import DummyClassifier, DummyRegressor
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.metrics import classification_report, mean_squared_error, accuracy_score
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import r2_score
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [3]:
df = pd.read_csv("df_co2_propre.csv")
df.head()

,BuildingType,PrimaryPropertyType,Latitude,Longitude,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,LargestPropertyUseType,ENERGYSTARScore,...,SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),DefaultData,TotalGHGEmissions,GHGEmissionsIntensity
0,NonResidential,Hotel,47.61220,-122.33799,1927,1.0,12,88434,Hotel,60.0,...,7226362.5,7456910.0,2003882.00,1.156514e+06,3946027.0,12764.52930,1276453.0,False,249.98,2.83
1,NonResidential,Hotel,47.61317,-122.33393,1996,1.0,11,103566,Hotel,61.0,...,8387933.0,8664479.0,0.00,9.504252e+05,3242851.0,51450.81641,5145082.0,False,295.86,2.86
2,NonResidential,Hotel,47.61393,-122.33810,1969,1.0,41,956110,Hotel,43.0,...,72587024.0,73937112.0,21566554.00,1.451544e+07,49526664.0,14938.00000,1493800.0,False,2089.28,2.19
3,NonResidential,Hotel,47.61412,-122.33664,1926,1.0,10,61320,Hotel,56.0,...,6794584.0,6946800.5,2214446.25,8.115253e+05,2768924.0,18112.13086,1811213.0,False,286.43,4.67
4,NonResidential,Hotel,47.61375,-122.34047,1980,1.0,18,175580,Hotel,75.0,...,14172606.0,14656503.0,0.00,1.573449e+06,5368607.0,88039.98438,8803998.0,False,505.01,2.88


In [5]:
df.dtypes

BuildingType               object
PrimaryPropertyType        object
Latitude                  float64
Longitude                 float64
YearBuilt                   int64
NumberofBuildings         float64
NumberofFloors              int64
PropertyGFATotal            int64
LargestPropertyUseType     object
ENERGYSTARScore           float64
SiteEUI(kBtu/sf)          float64
SiteEUIWN(kBtu/sf)        float64
SourceEUI(kBtu/sf)        float64
SourceEUIWN(kBtu/sf)      float64
SiteEnergyUse(kBtu)       float64
SiteEnergyUseWN(kBtu)     float64
SteamUse(kBtu)            float64
Electricity(kWh)          float64
Electricity(kBtu)         float64
NaturalGas(therms)        float64
NaturalGas(kBtu)          float64
DefaultData                  bool
TotalGHGEmissions         float64
GHGEmissionsIntensity     float64
dtype: object

In [28]:
cat_cols = ['BuildingType', 'PrimaryPropertyType', 'LargestPropertyUseType']
df[cat_cols] = df[cat_cols].astype('string')

# Afficher les types de données après conversion
print(df.dtypes)


BuildingType               string
PrimaryPropertyType        string
Latitude                  float64
Longitude                 float64
YearBuilt                   int64
NumberofBuildings         float64
NumberofFloors              int64
PropertyGFATotal            int64
LargestPropertyUseType     string
ENERGYSTARScore           float64
SiteEUI(kBtu/sf)          float64
SiteEUIWN(kBtu/sf)        float64
SourceEUI(kBtu/sf)        float64
SourceEUIWN(kBtu/sf)      float64
SiteEnergyUse(kBtu)       float64
SiteEnergyUseWN(kBtu)     float64
SteamUse(kBtu)            float64
Electricity(kWh)          float64
Electricity(kBtu)         float64
NaturalGas(therms)        float64
NaturalGas(kBtu)          float64
DefaultData                  bool
TotalGHGEmissions         float64
GHGEmissionsIntensity     float64
dtype: object


In [4]:
df.columns

Index(['BuildingType', 'PrimaryPropertyType', 'Latitude', 'Longitude',
       'YearBuilt', 'NumberofBuildings', 'NumberofFloors', 'PropertyGFATotal',
       'LargestPropertyUseType', 'ENERGYSTARScore', 'SiteEUI(kBtu/sf)',
       'SiteEUIWN(kBtu/sf)', 'SourceEUI(kBtu/sf)', 'SourceEUIWN(kBtu/sf)',
       'SiteEnergyUse(kBtu)', 'SiteEnergyUseWN(kBtu)', 'SteamUse(kBtu)',
       'Electricity(kWh)', 'Electricity(kBtu)', 'NaturalGas(therms)',
       'NaturalGas(kBtu)', 'DefaultData', 'TotalGHGEmissions',
       'GHGEmissionsIntensity'],
      dtype='object')

In [29]:
categ_features= ["BuildingType","PrimaryPropertyType","LargestPropertyUseType"]
numeric_features=["Latitude","Longitude","YearBuilt","NumberofBuildings","NumberofFloors","PropertyGFATotal",
"ENERGYSTARScore","SiteEUI(kBtu/sf)","SiteEUIWN(kBtu/sf)","SourceEUI(kBtu/sf)","SourceEUIWN(kBtu/sf)","SiteEnergyUse(kBtu)",
"SiteEnergyUseWN(kBtu)","SteamUse(kBtu)","Electricity(kWh)","Electricity(kBtu)","NaturalGas(therms)","NaturalGas(kBtu)",
"DefaultData","TotalGHGEmissions","GHGEmissionsIntensity"]


In [30]:
Y= df["TotalGHGEmissions"]

X=df.drop("TotalGHGEmissions", axis=1)

In [31]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

In [32]:
pipe_num = Pipeline(steps=[
    ("scaler" ,RobustScaler())
])

pipe_categ = Pipeline(steps=[
    ( "le", OneHotEncoder())
])

pipeline= ColumnTransformer(
    transformers=[
        ('pipenum',pipe_num, numeric_features),
        ("pipecat", pipe_categ, categ_features)]
    
)


In [35]:
df

,BuildingType,PrimaryPropertyType,Latitude,Longitude,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,LargestPropertyUseType,ENERGYSTARScore,...,SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),DefaultData,TotalGHGEmissions,GHGEmissionsIntensity
0,NonResidential,Hotel,47.61220,-122.33799,1927,1.0,12,88434,Hotel,60.0,...,7.226362e+06,7.456910e+06,2003882.00,1.156514e+06,3.946027e+06,12764.529300,1.276453e+06,False,249.98,2.83
1,NonResidential,Hotel,47.61317,-122.33393,1996,1.0,11,103566,Hotel,61.0,...,8.387933e+06,8.664479e+06,0.00,9.504252e+05,3.242851e+06,51450.816410,5.145082e+06,False,295.86,2.86
2,NonResidential,Hotel,47.61393,-122.33810,1969,1.0,41,956110,Hotel,43.0,...,7.258702e+07,7.393711e+07,21566554.00,1.451544e+07,4.952666e+07,14938.000000,1.493800e+06,False,2089.28,2.19
3,NonResidential,Hotel,47.61412,-122.33664,1926,1.0,10,61320,Hotel,56.0,...,6.794584e+06,6.946800e+06,2214446.25,8.115253e+05,2.768924e+06,18112.130860,1.811213e+06,False,286.43,4.67
4,NonResidential,Hotel,47.61375,-122.34047,1980,1.0,18,175580,Hotel,75.0,...,1.417261e+07,1.465650e+07,0.00,1.573449e+06,5.368607e+06,88039.984380,8.803998e+06,False,505.01,2.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3316,Nonresidential COS,Office,47.56722,-122.31154,1990,1.0,1,12294,Office,46.0,...,8.497457e+05,9.430032e+05,0.00,1.536550e+05,5.242709e+05,3254.750244,3.254750e+05,True,20.94,1.70
3317,Nonresidential COS,Other,47.59625,-122.32283,2004,1.0,1,16000,Other - Recreation,75.0,...,9.502762e+05,1.053706e+06,0.00,1.162210e+05,3.965461e+05,5537.299805,5.537300e+05,False,32.17,2.01
3318,Nonresidential COS,Other,47.63644,-122.35784,1974,1.0,1,13157,Other - Recreation,75.0,...,5.765898e+06,6.053764e+06,0.00,5.252517e+05,1.792159e+06,39737.390630,3.973739e+06,False,223.54,16.99
3319,Nonresidential COS,Mixed Use Property,47.52832,-122.32431,1989,1.0,1,14101,Other - Recreation,75.0,...,7.194712e+05,7.828413e+05,0.00,1.022480e+05,3.488702e+05,3706.010010,3.706010e+05,False,22.11,1.57


In [33]:
pipeline.fit(X_train, Y_train)

# Faire des prédictions sur l'ensemble de test
y_pred = pipeline.predict(X_test)

# Évaluer le modèle
mse = mean_squared_error(Y_test, Y_pred)
r2 = r2_score(Y_test, Y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

ValueError: A given column is not a column of the dataframe

In [43]:
df.drop("TotalGHGEmissions", axis=1)

,BuildingType,PrimaryPropertyType,Latitude,Longitude,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,LargestPropertyUseType,ENERGYSTARScore,...,SourceEUIWN(kBtu/sf),SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),DefaultData,GHGEmissionsIntensity
0,NonResidential,Hotel,47.61220,-122.33799,1927,1.0,12,88434,Hotel,60.0,...,189.000000,7.226362e+06,7.456910e+06,2003882.00,1.156514e+06,3.946027e+06,12764.529300,1.276453e+06,False,2.83
1,NonResidential,Hotel,47.61317,-122.33393,1996,1.0,11,103566,Hotel,61.0,...,179.399994,8.387933e+06,8.664479e+06,0.00,9.504252e+05,3.242851e+06,51450.816410,5.145082e+06,False,2.86
2,NonResidential,Hotel,47.61393,-122.33810,1969,1.0,41,956110,Hotel,43.0,...,244.100006,7.258702e+07,7.393711e+07,21566554.00,1.451544e+07,4.952666e+07,14938.000000,1.493800e+06,False,2.19
3,NonResidential,Hotel,47.61412,-122.33664,1926,1.0,10,61320,Hotel,56.0,...,224.000000,6.794584e+06,6.946800e+06,2214446.25,8.115253e+05,2.768924e+06,18112.130860,1.811213e+06,False,4.67
4,NonResidential,Hotel,47.61375,-122.34047,1980,1.0,18,175580,Hotel,75.0,...,215.600006,1.417261e+07,1.465650e+07,0.00,1.573449e+06,5.368607e+06,88039.984380,8.803998e+06,False,2.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3316,Nonresidential COS,Office,47.56722,-122.31154,1990,1.0,1,12294,Office,46.0,...,176.100006,8.497457e+05,9.430032e+05,0.00,1.536550e+05,5.242709e+05,3254.750244,3.254750e+05,True,1.70
3317,Nonresidential COS,Other,47.59625,-122.32283,2004,1.0,1,16000,Other - Recreation,75.0,...,118.900002,9.502762e+05,1.053706e+06,0.00,1.162210e+05,3.965461e+05,5537.299805,5.537300e+05,False,2.01
3318,Nonresidential COS,Other,47.63644,-122.35784,1974,1.0,1,13157,Other - Recreation,75.0,...,767.799988,5.765898e+06,6.053764e+06,0.00,5.252517e+05,1.792159e+06,39737.390630,3.973739e+06,False,16.99
3319,Nonresidential COS,Mixed Use Property,47.52832,-122.32431,1989,1.0,1,14101,Other - Recreation,75.0,...,110.800003,7.194712e+05,7.828413e+05,0.00,1.022480e+05,3.488702e+05,3706.010010,3.706010e+05,False,1.57


In [73]:
object_features = ["BuildingType", "PrimaryPropertyType", "LargestPropertyUseType"]
num_features = ["SiteEnergyUse(kBtu)", "SiteEnergyUseWN(kBtu)"]


In [71]:
object_features + num_features

['BuildingType',
 'PrimaryPropertyType',
 'LargestPropertyUseType',
 'SiteEnergyUse(kBtu)',
 'SiteEnergyUseWN(kBtu)']

In [72]:
object_features = ["BuildingType", "PrimaryPropertyType", "LargestPropertyUseType"]
num_features = ["SiteEnergyUse(kBtu)", "SiteEnergyUseWN(kBtu)"]

target = ["TotalGHGEmissions"]  # Remplacez par la colonne que vous souhaitez prédire

y = df[target]  # Remplacez par la colonne que vous souhaitez prédire

# y contenant object_features et num_features
X = df[object_features + num_features]


X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Créer le transformer pour les features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), object_features)
    ])

# Créer la pipeline avec le preprocessor et le modèle de régression linéaire
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Entraîner le modèle avec la pipeline
pipeline.fit(X_train, Y_train)

# Faire des prédictions sur l'ensemble de test
Y_pred = pipeline.predict(X_test)

# Évaluer le modèle
mse = mean_squared_error(Y_test, Y_pred)
r2 = r2_score(Y_test, Y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

Mean Squared Error: 46494.70023915456
R^2 Score: 0.9138889502856189


In [25]:
features = [
    "SiteEUI(kBtu/sf)", "SiteEUIWN(kBtu/sf)", "SourceEUI(kBtu/sf)", "SourceEUIWN(kBtu/sf)", 
    "SiteEnergyUse(kBtu)", "SiteEnergyUseWN(kBtu)", "TotalGHGEmissions", "GHGEmissionsIntensity"

]

# Extraire les features
X = df[features]

# Standardiser les données
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Calculer le VIF
vif_data = pd.DataFrame()
vif_data["feature"] = features
vif_data["VIF"] = [variance_inflation_factor(X_scaled, i) for i in range(X_scaled.shape[1])]

print(vif_data)

                 feature          VIF
0       SiteEUI(kBtu/sf)  1578.469886
1     SiteEUIWN(kBtu/sf)  1485.844839
2     SourceEUI(kBtu/sf)  1791.038415
3   SourceEUIWN(kBtu/sf)  1659.118346
4    SiteEnergyUse(kBtu)     5.418758
5  SiteEnergyUseWN(kBtu)     5.148214
6      TotalGHGEmissions    11.626797
7  GHGEmissionsIntensity     7.173870


In [76]:
features = [ 
    "SiteEnergyUse(kBtu)", "SiteEnergyUseWN(kBtu)", "TotalGHGEmissions"

]

# Extraire les features
X = df[features]

# Standardiser les données
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Calculer le VIF
vif_data = pd.DataFrame()
vif_data["feature"] = features
vif_data["VIF"] = [variance_inflation_factor(X, i) for i in range(X.shape[1])]

print(vif_data)

                 feature       VIF
0    SiteEnergyUse(kBtu)  4.131865
1  SiteEnergyUseWN(kBtu)  3.987899
2      TotalGHGEmissions  7.553540


In [79]:
object_features = ["BuildingType", "PrimaryPropertyType", "LargestPropertyUseType"]
num_features = ["PropertyGFATotal"]

target = ["TotalGHGEmissions"]  # Remplacez par la colonne que vous souhaitez prédire

y = df[target]  # Remplacez par la colonne que vous souhaitez prédire

# y contenant object_features et num_features
X = df[object_features + num_features]


X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Créer le transformer pour les features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), object_features)
    ])

# Créer la pipeline avec le preprocessor et le modèle de régression linéaire
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Entraîner le modèle avec la pipeline
pipeline.fit(X_train, Y_train)

# Faire des prédictions sur l'ensemble de test
Y_pred = pipeline.predict(X_test)

# Évaluer le modèle
mse = mean_squared_error(Y_test, Y_pred)
r2 = r2_score(Y_test, Y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

Mean Squared Error: 242190.54911519383
R^2 Score: 0.5514481799444128
